# Building a Spam Filter with Naive Bayes

In this project we will try to classify messages as spam or non-spam with the help of the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS.

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the The [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). You can also download the dataset directly from this [link](https://dq-content.s3.amazonaws.com/433/SMSSpamCollection).

Note that due to the nature of spam messages, the dataset contains content that may be offensive to some users.

## Exploring the Dataset

In [1]:
import pandas as pd
sms = pd.read_csv("SMSSpamCollection", sep = "\t", header = None, names=['Label', 'SMS'])
print("\n", "Rows:", sms.shape[0], "\n", "Columns:", sms.shape[1])
sms.head()


 Rows: 5572 
 Columns: 2


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [2]:
# spam and non-spam("ham") messages in %:
type_sms_percentages  = round(sms["Label"].value_counts(normalize = True)*100,2)
type_sms_percentages

ham     86.59
spam    13.41
Name: Label, dtype: float64

## Training and Test Set

We are going to make a test filter. Let's start with splitting our data in two categories:

- A **training set**, which we'll use to "train" the computer how to classify messages.


- A **test set**, which we'll use to test how good the spam filter is with classifying new messages.

We're going to keep 80% of our dataset for training, and 20% for testing. The dataset has 5,572 messages, which means that:

- The training set will have 4,458 messages (about 80% of the dataset).


- The test set will have 1,114 messages (about 20% of the dataset).

In [3]:
# randomizing the entire dataset:
sms_random = sms.sample(frac=1, random_state =1)

# splitting the entire dataset into train(80%) and test(20%) datasets:
# train dataset:
sms_train = sms_random.iloc[:4458]
sms_train.reset_index(inplace=True, drop = True)
train_spam_pt = round(sms_train["Label"].value_counts(normalize=True)*100,2)
print("Ham vs Spam Percentages in the Train Dataset:")
print(train_spam_pt)
print("\n")

# test dataset:
sms_test = sms_random.iloc[4458:]
sms_test.reset_index(inplace=True, drop = True)
test_spam_pt = round(sms_test["Label"].value_counts(normalize=True)*100,2)
print("Ham vs Spam Percentages in the Test Dataset:")
print(test_spam_pt)


Ham vs Spam Percentages in the Train Dataset:
ham     86.54
spam    13.46
Name: Label, dtype: float64


Ham vs Spam Percentages in the Test Dataset:
ham     86.8
spam    13.2
Name: Label, dtype: float64


We see that our distribution between spam and non-spam messages in both datasets do not much differ from the entire dataset. The same +- 13% for spam messages and +-86% for non-spam ("ham") sms.

## Letter Case and Punctuation

Below we will do some cleaning in order to change our sms and left only words without punctuation.


In [4]:
# removing the punctuation
import re
sms_train = sms_train.copy()
sms_train["SMS"] = sms_train["SMS"].apply(lambda x: re.sub(r"\W", " ", x)).str.lower()

#changes test
sms_train["SMS"].head(3)

0                     yep  by the pretty sculpture
1    yes  princess  are you going to make me moan 
2                       welp apparently he retired
Name: SMS, dtype: object

## Creating the Vocabulary

Below we will extract all unique words from "SMS" column and make the "Vocabulary" for training our algorithm.

In [5]:
sms_train["SMS"] = sms_train["SMS"].str.split() # remove whitespaces

vocabulary = []                                 # extract all the words
for sms in sms_train["SMS"]:
    for w in sms:
        vocabulary.append(w)

vocabulary = set(vocabulary)                    # remove duplicates
vocabulary = list(vocabulary)                   # return to list type
len(vocabulary)

7783

## The Final Training Set




In [6]:
# dictionary of unique words and count:
word_counts_per_sms = {unique_word: [0] * len(sms_train['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(sms_train['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

# new df from dictionary:       
new_sms_train = pd.DataFrame(word_counts_per_sms)

In [7]:
# concat 2 df:
training_set = pd.concat([sms_train, new_sms_train], axis=1)
training_set.head()

,Label,SMS,sk38xh,yhl,brown,sura,goto,bloomberg,vijay,nudist,...,should,side,flaked,awesome,popcorn,efreefone,fne,mmmmm,cherthala,wonders
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Calculating Constants First

We're now done with cleaning the training set, and we can begin creating the spam filter. The Naive Bayes algorithm will need to answer these two probability questions to be able to classify new messages:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam) \\
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation} 


Also, to calculate $ P(w_i|Spam) $ and $ P(w_i|Ham) $ inside the formulas above, we'll need to use these equations:



\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}



Some of the terms in the four equations above will have the same value for every new message. We can calculate the value of these terms once and avoid doing the computations again when a new messages comes in. Below, we'll use our training set to calculate:

- $P(Spam)$  and  $P(Ham)$


- $N_{Spam}, N_{Ham}, N_{Vocabulary}$


We'll also use Laplace smoothing and set  $\alpha = 1$


In [8]:
# Isolating spam and ham messages first
spam_messages = training_set[training_set['Label'] == 'spam']
ham_messages = training_set[training_set['Label'] == 'ham']

# P(Spam) and P(Ham)
p_spam = len(spam_messages) / len(training_set)
p_ham = len(ham_messages) / len(training_set)

# N_Spam
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()

# N_Ham
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()

# N_Vocabulary
n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1

## Calculating Parameters

Now that we have the constant terms calculated above, we can move on with calculating the parameters $P(w_i|Spam)$
 and $P(w_i|Ham)$
. Each parameter will thus be a conditional probability value associated with each word in the vocabulary.

The parameters are calculated using the formulas:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

In [9]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# Calculate parameters
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum()   # spam_messages already defined in a cell above
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham_messages[word].sum()   # ham_messages already defined in a cell above
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

## Classifying a New Message



In [10]:
# new message classifying function
def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
            
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)
    
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [11]:
# test 1
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [12]:
# test 2
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


## Measuring the Spam Filter's Accurancy 

We'll now try to determine how well the spam filter does on our test set of 1,114 messages.

First off, we'll change the `classify()` function that we wrote previously to return the labels instead of printing them.



In [13]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]

        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

Now that we have a function that returns labels instead of printing them, we can use it to create a new column in our test set.

In [14]:
sms_test = sms_test.copy()
sms_test['predicted'] = sms_test['SMS'].apply(classify_test_set)
sms_test.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Now we can compare the predicted values with the actual values to measure how good our spam filter is with classifying new messages. To make the measurement, we'll use **accuracy** as a metric:

\begin{equation}
\text{Accuracy} = \frac{\text{number of correctly classified messages}}{\text{total number of classified messages}}
\end{equation}

In [15]:
correct = 0
total = sms_test.shape[0]

for row in sms_test.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1

print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 1100
Incorrect: 14
Accuracy: 0.9874326750448833


The accuracy is close to 98.74%, which is really good. Our spam filter looked at 1,114 messages that it hasn't seen in training, and classified 1,100 correctly.

### Conclusion

We managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter had an accuracy of 98.74% on the test set we used, which is a pretty good result. Our initial goal was an accuracy of over 80%, and we managed to do way better than that.